In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
# computing devices

def cpu():
    return torch.device('cpu')

def gpu(i=0):
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

In [3]:
def num_gpus():
    return torch.cuda.device_count()

num_gpus()

1

In [4]:
## Functions to run code even if the requested GPUs do not exist

def try_gpu(i=0):
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [5]:
X = torch.ones(2, 3, device=try_gpu())
X #Storing in the gpu

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [6]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y # It would store Y in cuda:1 if you have two gpus

tensor([[0.1677, 0.0526, 0.3338],
        [0.2759, 0.0384, 0.0472]])

In [7]:
Y.device

device(type='cpu')

If you have variables in two gpus and want to perform calculation, then you can transfer one of them to another gpu and perform calculations. But transfering data is very slow and it is not prefered. It is much slower than computation. 

In [8]:
## Neural Network and GPUs

net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

net(X)

/home/drishya/miniconda3/envs/d2l/lib/python3.9/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


tensor([[-0.4942],
        [-0.4942]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [9]:
## Checking to ensure model parameters are stored on the same gpu
net[0].weight.data.device

device(type='cuda', index=0)

In [10]:
@d2l.add_to_class(d2l.Trainer)
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch] # moves each element in the batch to the first GPU
    return batch

@d2l.add_to_class(d2l.Trainer)
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model